In [ ]:
!pip install torchio opencv-python-headless matplotlib prefetch_generator monai edt surface-distance medim
!pip install nilearn scikit-image open3d scipy numpy

In [1]:
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d

def normalize_to_unit_cube(points):
    """
    Normalizes a point cloud independently into the range [-1, 1] 
    as described in Section D.3.1.
    
    Args:
        points (np.ndarray): (N, 3) array of points.
        
    Returns:
        np.ndarray: Normalized points.
    """
    # Center the points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    
    # Scale to fit in [-1, 1]
    # We find the maximum absolute distance from origin along any axis
    max_dist = np.max(np.abs(points_centered))
    
    # Scale such that the bounding box is within [-1, 1]
    if max_dist > 0:
        points_normalized = points_centered / max_dist
    else:
        points_normalized = points_centered
        
    return points_normalized

def apply_icp(source_points, target_points, threshold=0.02):
    """
    Applies Iterative Closest Point (ICP) alignment.
    
    Args:
        source_points (np.ndarray): Predicted points (to be aligned).
        target_points (np.ndarray): Ground truth points (reference).
        threshold (float): Max correspondence distance for ICP.
        
    Returns:
        np.ndarray: The aligned source points.
    """
    # Convert to Open3D point clouds
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    # Initial alignment (Identity)
    trans_init = np.identity(4)
    
    # Apply Point-to-Point ICP
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    
    # Transform the source points
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score_at_threshold(pred_points, gt_points, threshold=0.01):
    """
    Computes F1 score at a specific threshold.
    
    Formula:
    Precision = % of pred points within threshold of any gt point
    Recall = % of gt points within threshold of any pred point
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    
    Args:
        pred_points (np.ndarray): (N, 3) array of predicted points.
        gt_points (np.ndarray): (M, 3) array of ground truth points.
        threshold (float): Distance threshold (default 0.01 as per paper).
        
    Returns:
        dict: Contains 'f1', 'precision', and 'recall'.
    """
    # Build KD-Trees for efficient nearest neighbor search
    # This is much faster than brute force distance calculation for 1M points
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    # --- Compute Precision (Pred -> GT) ---
    # For each point in Pred, find distance to nearest neighbor in GT
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    
    # Precision: Percentage of pred points close enough to GT
    precision = np.mean(dist_pred_to_gt < threshold)
    
    # --- Compute Recall (GT -> Pred) ---
    # For each point in GT, find distance to nearest neighbor in Pred
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    
    # Recall: Percentage of GT points close enough to Pred
    recall = np.mean(dist_gt_to_pred < threshold)
    
    # --- Compute F1 Score (Harmonic Mean) ---
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
        
    return {
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

def evaluate_sam3d_metric(pred_points_raw, gt_points_raw):
    """
    Full pipeline wrapper: Normalization -> ICP -> F1@0.01
    """
    # 1. Normalize independently into [-1, 1]
    pred_norm = normalize_to_unit_cube(pred_points_raw)
    gt_norm = normalize_to_unit_cube(gt_points_raw)
    
    # 2. Apply ICP Alignment
    # Note: ICP usually requires a rough initial alignment. 
    # Since we normalized to the same unit cube, they should be roughly at the same scale/center.
    # However, rotations might still differ. This step handles fine-tuning.
    pred_aligned = apply_icp(pred_norm, gt_norm)
    
    # 3. Compute Metrics
    metrics = compute_f1_score_at_threshold(pred_aligned, gt_norm, threshold=0.01)
    
    return metrics

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
import numpy as np
import open3d as o3d
from nilearn import image
from skimage import measure
from scipy.spatial import cKDTree

# ==========================================
# 1. Previous Metric Code (from previous answer)
# ==========================================

def normalize_to_unit_cube(points):
    """
    Normalizes a point cloud independently into the range [-1, 1] 
    as described in Section D.3.1.
    """
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    if max_dist > 0:
        points_normalized = points_centered / max_dist
    else:
        points_normalized = points_centered
    return points_normalized

def apply_icp(source_points, target_points, threshold=0.02):
    """
    Applies ICP alignment as required by the methodology.
    """
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    """
    Computes F1 score using Precision and Recall.
    """
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    # Precision: % of pred points within threshold of GT
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    # Recall: % of GT points within threshold of pred
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
        
    return f1, precision, recall

# ==========================================
# 2. New NIfTI Handling Code
# ==========================================

def nii_to_surface_points(nii_path, num_points=1000000, iso_value=0.5):
    """
    Reads a NIfTI file, extracts the isosurface using Marching Cubes,
    and uniformly samples points from that surface.
    
    Args:
        nii_path (str): Path to the .nii.gz file.
        num_points (int): Number of points to sample (Paper uses 1M ).
        iso_value (float): Threshold for binary mask extraction.
        
    Returns:
        np.ndarray: (N, 3) array of surface points in world coordinates.
    """
    # 1. Load NIfTI file
    img = image.load_img(nii_path)
    data = img.get_fdata()
    affine = img.affine

    # 2. Extract Surface Mesh (Marching Cubes)
    # This converts voxel data to a triangulated mesh (verts, faces)
    # Step=1 ensures we check every voxel.
    try:
        verts_voxel, faces, _, _ = measure.marching_cubes(data, level=iso_value, step_size=1)
    except ValueError:
        print(f"Error: No surface found in {nii_path} (is the image empty?)")
        return np.zeros((0, 3))

    # 3. Apply Affine Transform (Voxel -> World Coordinates)
    # We must respect voxel spacing (e.g., anisotropy) for accurate shape comparison.
    # Add homogeneous coordinate (column of 1s)
    ones = np.ones((verts_voxel.shape[0], 1))
    verts_homo = np.hstack([verts_voxel, ones])
    
    # Multiply by affine matrix (transpose is needed for matrix multiplication order here)
    verts_world = verts_homo @ affine.T
    verts_world = verts_world[:, :3] # Drop homogeneous coordinate

    # 4. Create Open3D Mesh for Uniform Sampling
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(verts_world)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    
    # Ensure mesh is clean for sampling
    mesh.compute_vertex_normals()
    
    # 5. Uniformly Sample Points
    # This samples points based on face area, ensuring uniform coverage
    pcd = mesh.sample_points_uniformly(number_of_points=num_points)
    
    return np.asarray(pcd.points)

def evaluate_nifti_pair(pred_path, gt_path):
    print(f"Processing Prediction: {pred_path}")
    pred_points_raw = nii_to_surface_points(pred_path)
    
    print(f"Processing Ground Truth: {gt_path}")
    gt_points_raw = nii_to_surface_points(gt_path)
    
    if len(pred_points_raw) == 0 or len(gt_points_raw) == 0:
        print("Failed to extract surfaces.")
        return

    # --- Apply Paper Methodology ---
    
    # 1. Normalize independently [-1, 1] 
    pred_norm = normalize_to_unit_cube(pred_points_raw)
    gt_norm = normalize_to_unit_cube(gt_points_raw)
    
    # 2. Apply ICP Alignment 
    # (Threshold 0.05 allows for initial convergence, can be tightened)
    pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
    
    # 3. Compute Metrics (Threshold 0.01) [cite: 1152]
    f1, prec, rec = compute_f1_score(pred_aligned, gt_norm, threshold=0.01)
    
    print("-" * 30)
    print(f"Evaluation Results:")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1@0.01:   {f1:.4f}")
    print("-" * 30)


scan_path = 'test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz'
mask_path = 'test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz'

evaluate_nifti_pair(scan_path, mask_path)

Processing Prediction: test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz
Processing Ground Truth: test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz


In [ ]:
import numpy as np
import nilearn.image
from skimage.measure import marching_cubes
import trimesh
from scipy.spatial import cKDTree
import open3d as o3d

# --- 1. Reuse the Metric Code from the Previous Step ---

def normalize_to_unit_cube(points):
    """Normalizes points to [-1, 1] as per[cite: 1149]."""
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    if max_dist > 0:
        return points_centered / max_dist
    return points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns point clouds using ICP[cite: 1150]."""
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score_at_threshold(pred_points, gt_points, threshold=0.01):
    """Computes harmonic mean of precision and recall[cite: 1152]."""
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
        
    return {"f1": f1, "precision": precision, "recall": recall}

# --- 2. New Code to Handle NIfTI Files ---

def nifti_to_pointcloud(nii_path, num_samples=1_000_000, level=0.5):
    """
    Reads a NIfTI file, extracts the surface mesh using marching cubes,
    and uniformly samples points from that surface.
    """
    # Load the image using nilearn
    print(f"Loading {nii_path}...")
    img = nilearn.image.load_img(nii_path)
    data = img.get_fdata()
    
    # Ensure data is suitable for marching cubes (binary mask preferred)
    # If the NIfTI contains probabilities, threshold it.
    if np.max(data) <= 1.0 and np.max(data) > 0:
         # Standardize mask to boolean
        data = data > level
    
    # Extract surface mesh (vertices and faces)
    # marching_cubes returns: verts, faces, normals, values
    try:
        verts, faces, _, _ = marching_cubes(data, level=level)
    except RuntimeError:
        print(f"Warning: No surface found in {nii_path} at level {level}.")
        return np.zeros((num_samples, 3))

    # Apply the affine transformation to map voxel coords to real-world coords
    # NIfTI affine is a 4x4 matrix
    affine = img.affine
    # Homogeneous coordinates for vertices
    verts_homogeneous = np.c_[verts, np.ones(verts.shape[0])]
    # Apply affine: x_new = affine @ x_old
    verts_transformed = (affine @ verts_homogeneous.T).T[:, :3]

    # Create a Trimesh object for easy sampling
    mesh = trimesh.Trimesh(vertices=verts_transformed, faces=faces)
    
    # Uniformly sample points from the mesh surface 
    samples, _ = trimesh.sample.sample_surface(mesh, num_samples)
    
    return samples

def evaluate_nifti_pair(pred_path, gt_path):
    """
    Full pipeline to evaluate two NIfTI files.
    """
    # 1. Convert NIfTI volumes to Surface Point Clouds
    # The paper specifies sampling 1M points 
    N = 1_000_000 
    
    print("Converting prediction volume to surface points...")
    pred_points = nifti_to_pointcloud(pred_path, num_samples=N)
    
    print("Converting ground truth volume to surface points...")
    gt_points = nifti_to_pointcloud(gt_path, num_samples=N)
    
    # Check if we got valid points
    if len(pred_points) == 0 or len(gt_points) == 0:
        print("Error: Empty point cloud generated.")
        return None

    # 2. Normalize [cite: 1149]
    print("Normalizing...")
    pred_norm = normalize_to_unit_cube(pred_points)
    gt_norm = normalize_to_unit_cube(gt_points)
    
    # 3. Align [cite: 1150]
    print("Aligning via ICP...")
    pred_aligned = apply_icp(pred_norm, gt_norm)
    
    # 4. Compute Metrics [cite: 1152]
    print("Computing F1@0.01...")
    metrics = compute_f1_score_at_threshold(pred_aligned, gt_norm, threshold=0.01)
    
    return metrics

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    """
    Computes Voxel-IoU by voxelizing point clouds into a grid of specified resolution.
    
    Paper Description: "We voxelize both point clouds to 64^3 resolution and 
    [cite_start]compute intersection-over-union over occupied voxels." [cite: 1155-1156]
    
    Args:
        point_cloud_1 (np.ndarray): (N, 3) aligned/normalized points [-1, 1].
        point_cloud_2 (np.ndarray): (M, 3) aligned/normalized points [-1, 1].
        resolution (int): Grid resolution (default 64).
        
    Returns:
        float: Voxel-IoU score.
    """
    # Define the voxel grid range. Since points are normalized to [-1, 1],
    # the grid should cover this range.
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    
    # Calculate voxel size
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_occupied_voxels(points):
        # 1. Shift points to [0, 2] range (relative to min_bound)
        points_shifted = points - min_bound
        
        # 2. Divide by voxel size to get fractional indices
        indices_float = points_shifted / voxel_size
        
        # 3. Floor to get integer indices
        indices = np.floor(indices_float).astype(int)
        
        # 4. Clip indices to ensure they are within [0, resolution-1]
        # (Handles potential floating point errors at the exact boundaries)
        indices = np.clip(indices, 0, resolution - 1)
        
        # 5. Get unique indices (set of occupied voxels)
        # We turn the Nx3 array into a set of tuples for efficient set operations
        occupied_set = set(map(tuple, indices))
        return occupied_set

    # Get sets of occupied voxel coordinates for both shapes
    voxels_1 = get_occupied_voxels(point_cloud_1)
    voxels_2 = get_occupied_voxels(point_cloud_2)
    
    # Compute Intersection over Union
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    
    if union == 0:
        return 0.0
        
    return intersection / union

# ==========================================
# Updated Evaluation Function
# ==========================================

def evaluate_nifti_pair_complete(pred_path, gt_path):
    print(f"Processing Prediction: {pred_path}")
    pred_points_raw = nii_to_surface_points(pred_path)
    
    print(f"Processing Ground Truth: {gt_path}")
    gt_points_raw = nii_to_surface_points(gt_path)
    
    if len(pred_points_raw) == 0 or len(gt_points_raw) == 0:
        print("Failed to extract surfaces.")
        return

    # 1. Normalize independently [-1, 1]
    pred_norm = normalize_to_unit_cube(pred_points_raw)
    gt_norm = normalize_to_unit_cube(gt_points_raw)
    
    # 2. Apply ICP Alignment
    print("Aligning meshes...")
    pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
    
    # 3. Compute Metrics
    print("Computing F1 Score...")
    f1, prec, rec = compute_f1_score(pred_aligned, gt_norm, threshold=0.01)
    
    print("Computing Voxel-IoU...")
    # Note: We use the *aligned* prediction and the *normalized* ground truth
    # because Voxel-IoU is sensitive to alignment errors.
    iou = compute_voxel_iou(pred_aligned, gt_norm, resolution=64)
    
    print("-" * 30)
    print(f"Evaluation Results:")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1@0.01:   {f1:.4f}")
    print(f"Voxel-IoU: {iou:.4f}")
    print("-" * 30)

# --- Example Usage ---
scan_path = 'test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz'
mask_path = 'test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz'

evaluate_nifti_pair_complete(scan_path, mask_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Converting prediction volume to surface points...
Loading test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz...
Converting ground truth volume to surface points...
Loading test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz...
Normalizing...
Aligning via ICP...
Computing F1@0.01...


{'f1': np.float64(0.060535171385332084),
 'precision': np.float64(0.033162),
 'recall': np.float64(0.346783)}

In [ ]:
import numpy as np
import open3d as o3d
from nilearn import image
from skimage import measure
from scipy.spatial import cKDTree

# ==========================================
# 1. Metric Helper Functions (Unchanged)
# ==========================================

def normalize_to_unit_cube(points):
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
        
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    if len(pred_points) == 0 or len(gt_points) == 0:
        return 0.0, 0.0, 0.0
        
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0:
        return 0.0, precision, recall
        
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0 # Both empty
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0  # One empty
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    
    return intersection / union if union > 0 else 0.0

# ==========================================
# 2. Updated Multi-Label NIfTI Handling
# ==========================================

def extract_surface_for_label(data, affine, label_id, num_points=100000):
    """
    Extracts surface points SPECIFICALLY for a given integer label ID.
    """
    # Create binary mask for just this object
    # Note: 'level=0.5' on a boolean mask works to extract the boundary
    binary_mask = (data == label_id).astype(float)
    
    if np.sum(binary_mask) == 0:
        return np.zeros((0, 3))

    try:
        # Extract mesh from the binary mask of this specific label
        verts_voxel, faces, _, _ = measure.marching_cubes(binary_mask, level=0.5, step_size=1)
        
        # Apply affine transform to world coordinates
        ones = np.ones((verts_voxel.shape[0], 1))
        verts_world = (np.hstack([verts_voxel, ones]) @ affine.T)[:, :3]

        # Sample points
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(verts_world)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        mesh.compute_vertex_normals()
        
        pcd = mesh.sample_points_uniformly(number_of_points=num_points)
        return np.asarray(pcd.points)
        
    except (ValueError, RuntimeError):
        # Can happen if object is too small (single voxel)
        return np.zeros((0, 3))

def evaluate_multi_object_nifti(pred_path, gt_path):
    print(f"Loading files...")
    pred_img = image.load_img(pred_path)
    gt_img = image.load_img(gt_path)
    
    pred_data = pred_img.get_fdata()
    gt_data = gt_img.get_fdata()
    
    # Identify all unique object IDs in Ground Truth (excluding background 0)
    # We assume we want to evaluate every object present in the GT.
    gt_labels = np.unique(gt_data)
    gt_labels = gt_labels[gt_labels != 0].astype(int)
    
    print(f"Found {len(gt_labels)} objects in Ground Truth: {gt_labels}")
    
    results = {
        "f1": [],
        "iou": [],
        "precision": [],
        "recall": []
    }
    
    # Iterate over each object (1 to N)
    for label_id in gt_labels:
        print(f"\n--- Evaluating Object ID: {label_id} ---")
        
        # 1. Extract Surfaces for this specific ID
        # Note: We use the SAME label_id for pred and gt, assuming the
        # segmentation classes are consistent.
        gt_points = extract_surface_for_label(gt_data, gt_img.affine, label_id)
        pred_points = extract_surface_for_label(pred_data, pred_img.affine, label_id)
        
        if len(gt_points) == 0:
            print(f"Skipping Object {label_id} (GT empty/too small)")
            continue
            
        if len(pred_points) == 0:
            print(f"Object {label_id} missing in prediction!")
            results["f1"].append(0.0)
            results["iou"].append(0.0)
            results["precision"].append(0.0)
            results["recall"].append(0.0)
            continue
            
        # 2. Normalize & Align (Per-Object)
        gt_norm = normalize_to_unit_cube(gt_points)
        pred_norm = normalize_to_unit_cube(pred_points)
        
        # Align this specific object
        pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
        
        # 3. Compute Metrics for this object
        f1, prec, rec = compute_f1_score(pred_aligned, gt_norm, threshold=0.01)
        iou = compute_voxel_iou(pred_aligned, gt_norm)
        
        print(f"  F1@0.01:   {f1:.4f}")
        print(f"  Voxel-IoU: {iou:.4f}")
        
        results["f1"].append(f1)
        results["iou"].append(iou)
        results["precision"].append(prec)
        results["recall"].append(rec)
    
    # === Aggregate Results ===
    print("\n" + "="*30)
    print(f"FINAL AGGREGATE RESULTS (Mean over {len(results['f1'])} objects)")
    print(f"Mean F1@0.01:   {np.mean(results['f1']):.4f}")
    print(f"Mean Voxel-IoU: {np.mean(results['iou']):.4f}")
    print("="*30)

# --- Example Usage ---
scan_path = 'test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz'
mask_path = 'test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz'

evaluate_multi_object_nifti(scan_path, mask_path)

Loading files...
Found 15 objects in Ground Truth: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]

--- Evaluating Object ID: 1 ---
  F1@0.01:   0.0289
  Voxel-IoU: 0.0514

--- Evaluating Object ID: 2 ---
  F1@0.01:   0.0236
  Voxel-IoU: 0.0227

--- Evaluating Object ID: 3 ---
  F1@0.01:   0.0541
  Voxel-IoU: 0.0882

--- Evaluating Object ID: 4 ---
  F1@0.01:   0.0363
  Voxel-IoU: 0.0535

--- Evaluating Object ID: 5 ---
  F1@0.01:   0.0539
  Voxel-IoU: 0.0605

--- Evaluating Object ID: 6 ---
  F1@0.01:   0.0687
  Voxel-IoU: 0.0794

--- Evaluating Object ID: 7 ---
  F1@0.01:   0.0439
  Voxel-IoU: 0.0627

--- Evaluating Object ID: 8 ---
  F1@0.01:   0.0538
  Voxel-IoU: 0.0318

--- Evaluating Object ID: 9 ---
  F1@0.01:   0.0613
  Voxel-IoU: 0.0353

--- Evaluating Object ID: 10 ---
  F1@0.01:   0.0612
  Voxel-IoU: 0.0714

--- Evaluating Object ID: 11 ---
  F1@0.01:   0.0585
  Voxel-IoU: 0.0652

--- Evaluating Object ID: 12 ---
  F1@0.01:   0.0532
  Voxel-IoU: 0.0763

--- Evaluating Object 

In [ ]:
import numpy as np
import open3d as o3d
from nilearn import image
from skimage import measure
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

# ==========================================
# 1. Metric Helper Functions
# ==========================================

def normalize_to_unit_cube(points):
    """Normalizes point cloud to [-1, 1] range."""
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns source points to target points using ICP."""
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
    
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    """Computes F1@0.01 score."""
    if len(pred_points) == 0 or len(gt_points) == 0: return 0.0, 0.0, 0.0
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0: return 0.0, precision, recall
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-IoU at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    return intersection / union if union > 0 else 0.0

def compute_chamfer_distance(pred_points, gt_points):
    """Computes Chamfer Distance (CD)."""
    if len(pred_points) == 0 or len(gt_points) == 0: return float('inf')
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    
    return np.mean(dist_pred_to_gt) + np.mean(dist_gt_to_pred)

def compute_emd(pred_points, gt_points, max_points=2048):
    """
    Computes Earth Mover's Distance (EMD).
    
    Paper Definition: "Quantifies the minimal cost required to transport one point 
    distribution to match the other... enforcing bijective correspondence" [cite: 1158-1159].
    
    Note: Exact EMD is O(N^3). We downsample to `max_points` to keep this tractable on CPU.
    """
    if len(pred_points) == 0 or len(gt_points) == 0:
        return float('inf')

    # 1. Downsample for tractability
    # EMD requires |S1| == |S2| for a bijection. We assume equal resampling.
    n_points = min(len(pred_points), len(gt_points), max_points)
    
    # Random choice without replacement to maintain distribution properties
    # (Setting seed for reproducibility)
    np.random.seed(42)
    indices_pred = np.random.choice(len(pred_points), n_points, replace=False)
    indices_gt = np.random.choice(len(gt_points), n_points, replace=False)
    
    pred_sampled = pred_points[indices_pred]
    gt_sampled = gt_points[indices_gt]
    
    # 2. Compute Distance Matrix (Cost Matrix)
    # Shape: (N, N)
    dists = cdist(pred_sampled, gt_sampled, metric='euclidean')
    
    # 3. Solve Linear Sum Assignment (Hungarian Algorithm)
    # Finds row_ind, col_ind such that sum(dists[row, col]) is minimized
    row_ind, col_ind = linear_sum_assignment(dists)
    
    # 4. Compute Mean Cost
    # "Quantifies the minimal cost" [cite: 1158] -> usually averaged by N
    total_cost = dists[row_ind, col_ind].sum()
    emd_value = total_cost / n_points
    
    return emd_value

# ==========================================
# 2. Multi-Label Evaluation Pipeline
# ==========================================

def extract_surface_for_label(data, affine, label_id, num_points=100000):
    """Extracts surface points for a specific label ID."""
    binary_mask = (data == label_id).astype(float)
    if np.sum(binary_mask) == 0: return np.zeros((0, 3))

    try:
        verts_voxel, faces, _, _ = measure.marching_cubes(binary_mask, level=0.5, step_size=1)
        ones = np.ones((verts_voxel.shape[0], 1))
        verts_world = (np.hstack([verts_voxel, ones]) @ affine.T)[:, :3]
        
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(verts_world)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        mesh.compute_vertex_normals()
        
        pcd = mesh.sample_points_uniformly(number_of_points=num_points)
        return np.asarray(pcd.points)
    except (ValueError, RuntimeError):
        return np.zeros((0, 3))

def evaluate_all_metrics(pred_path, gt_path):
    print(f"Loading files...")
    pred_img = image.load_img(pred_path)
    gt_img = image.load_img(gt_path)
    
    pred_data = pred_img.get_fdata()
    gt_data = gt_img.get_fdata()
    
    gt_labels = np.unique(gt_data)
    gt_labels = gt_labels[gt_labels != 0].astype(int)
    
    results = {"f1": [], "iou": [], "chamfer": [], "emd": []}
    
    for label_id in gt_labels:
        print(f"\n--- Object ID: {label_id} ---")
        
        # 1. Extraction (High resolution for F1/CD)
        gt_points = extract_surface_for_label(gt_data, gt_img.affine, label_id, num_points=10000)
        pred_points = extract_surface_for_label(pred_data, pred_img.affine, label_id, num_points=10000)
        
        # Handle Missing/Empty
        if len(gt_points) == 0: continue
        if len(pred_points) == 0:
            results["f1"].append(0.0)
            results["iou"].append(0.0)
            results["chamfer"].append(1.0) # Penalty
            results["emd"].append(1.0)     # Penalty
            continue
            
        # 2. Normalize & Align
        gt_norm = normalize_to_unit_cube(gt_points)
        pred_norm = normalize_to_unit_cube(pred_points)
        pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
        
        # 3. Compute Metrics
        f1, _, _ = compute_f1_score(pred_aligned, gt_norm)
        iou = compute_voxel_iou(pred_aligned, gt_norm)
        chamfer = compute_chamfer_distance(pred_aligned, gt_norm)
        
        # EMD (Uses internal downsampling to e.g., 2048 points)
        emd = compute_emd(pred_aligned, gt_norm, max_points=2048)
        
        print(f"  F1@0.01:   {f1:.4f}")
        print(f"  Voxel-IoU: {iou:.4f}")
        print(f"  Chamfer:   {chamfer:.4f}")
        print(f"  EMD:       {emd:.4f}")
        
        results["f1"].append(f1)
        results["iou"].append(iou)
        results["chamfer"].append(chamfer)
        results["emd"].append(emd)
    
    print("\n" + "="*30)
    print(f"FINAL MEAN SCORES")
    print(f"Mean F1@0.01:   {np.mean(results['f1']):.4f}")
    print(f"Mean Voxel-IoU: {np.mean(results['iou']):.4f}")
    print(f"Mean Chamfer:   {np.mean(results['chamfer']):.4f}")
    print(f"Mean EMD:       {np.mean(results['emd']):.4f}")
    print("="*30)

# --- Example Usage ---
scan_path = 'test_data/amos_val_toy_data/imagesVa/amos_0013.nii.gz'
mask_path = 'test_data/amos_val_toy_data/labelsVa/amos_0013.nii.gz'

evaluate_all_metrics(scan_path, mask_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name 'evaluate_multi_object_nifti' is not defined

In [ ]:
import numpy as np
import open3d as o3d
from nilearn import image
from skimage import measure
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import os

# ==========================================
# 1. Metric Helper Functions (Unchanged)
# ==========================================

def normalize_to_unit_cube(points):
    """Normalizes point cloud to [-1, 1] range."""
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns source points to target points using ICP."""
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
    
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    """Computes F1@0.01 score."""
    if len(pred_points) == 0 or len(gt_points) == 0: return 0.0, 0.0, 0.0
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0: return 0.0, precision, recall
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-IoU at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    return intersection / union if union > 0 else 0.0

def compute_chamfer_distance(pred_points, gt_points):
    """Computes Chamfer Distance (CD)."""
    if len(pred_points) == 0 or len(gt_points) == 0: return float('inf')
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    
    return np.mean(dist_pred_to_gt) + np.mean(dist_gt_to_pred)

def compute_emd(pred_points, gt_points, max_points=2048):
    """Computes Earth Mover's Distance (EMD)."""
    if len(pred_points) == 0 or len(gt_points) == 0:
        return float('inf')

    n_points = min(len(pred_points), len(gt_points), max_points)
    
    np.random.seed(42)
    indices_pred = np.random.choice(len(pred_points), n_points, replace=False)
    indices_gt = np.random.choice(len(gt_points), n_points, replace=False)
    
    pred_sampled = pred_points[indices_pred]
    gt_sampled = gt_points[indices_gt]
    
    dists = cdist(pred_sampled, gt_sampled, metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(dists)
    
    total_cost = dists[row_ind, col_ind].sum()
    emd_value = total_cost / n_points
    
    return emd_value

# ==========================================
# 2. Data Extraction Helpers (Modified)
# ==========================================

def extract_surface_from_nii(nii_path, label_id, num_points=10000):
    """Extracts surface points from NIfTI using Marching Cubes."""
    img = image.load_img(nii_path)
    data = img.get_fdata()
    affine = img.affine
    
    binary_mask = (data == label_id).astype(float)
    if np.sum(binary_mask) == 0: 
        print(f"Warning: Label {label_id} not found in NIfTI.")
        return np.zeros((0, 3))

    try:
        # Extract surface mesh
        verts_voxel, faces, _, _ = measure.marching_cubes(binary_mask, level=0.5, step_size=1)
        # Apply affine to move to world coordinates
        ones = np.ones((verts_voxel.shape[0], 1))
        verts_world = (np.hstack([verts_voxel, ones]) @ affine.T)[:, :3]
        
        # Convert to Open3D to sample points uniformly
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(verts_world)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        mesh.compute_vertex_normals()
        
        pcd = mesh.sample_points_uniformly(number_of_points=num_points)
        return np.asarray(pcd.points)
    except (ValueError, RuntimeError) as e:
        print(f"Extraction Error: {e}")
        return np.zeros((0, 3))

def extract_surface_from_ply(ply_path, num_points=10000):
    """Loads a PLY file and samples points uniformly."""
    if not os.path.exists(ply_path):
        print(f"Error: PLY file not found at {ply_path}")
        return np.zeros((0, 3))
        
    try:
        # Try loading as a Mesh first (most likely for surfaces)
        mesh = o3d.io.read_triangle_mesh(ply_path)
        if len(mesh.triangles) > 0:
            pcd = mesh.sample_points_uniformly(number_of_points=num_points)
            return np.asarray(pcd.points)
        else:
            # If no triangles, load as PointCloud
            pcd = o3d.io.read_point_cloud(ply_path)
            # If point cloud is huge, downsample; if small, return as is
            # Ideally, we resample to num_points to match density
            if len(pcd.points) > 0:
                # Simple random downsampling if too large
                if len(pcd.points) > num_points:
                    pcd = pcd.random_down_sample(num_points / len(pcd.points))
                return np.asarray(pcd.points)
            return np.zeros((0, 3))
            
    except Exception as e:
        print(f"PLY Loading Error: {e}")
        return np.zeros((0, 3))

# ==========================================
# 3. Main Comparison Function
# ==========================================

def evaluate_nii_vs_ply(nii_path, ply_path, target_label_id=1):
    """
    Compares a specific object in a NIfTI file against a PLY mesh.
    
    Args:
        nii_path: Path to segmentation mask (.nii.gz)
        ply_path: Path to surface mesh (.ply)
        target_label_id: The integer label in the NIfTI file to compare.
    """
    print(f"Comparing NIfTI Label {target_label_id} vs PLY file...")
    
    # 1. Extraction
    print("Extracting NIfTI surface...")
    nii_points = extract_surface_from_nii(nii_path, target_label_id, num_points=10000)
    
    print("Loading PLY surface...")
    ply_points = extract_surface_from_ply(ply_path, num_points=10000)
    
    # Handle Missing/Empty
    if len(nii_points) == 0 or len(ply_points) == 0:
        print("Error: One of the inputs yielded 0 points.")
        return
        
    # 2. Normalize & Align
    # Assuming PLY is Ground Truth (GT) and NIfTI is Prediction (or vice versa)
    # We normalize both to unit cube to make metrics scale-invariant
    gt_norm = normalize_to_unit_cube(ply_points)
    pred_norm = normalize_to_unit_cube(nii_points)
    
    # Apply ICP to handle slight rotational/translational misalignments
    # warning: if coordinate systems are completely different (e.g. RAS vs LPS),
    # ICP might not be enough.
    pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
    
    # 3. Compute Metrics
    print("\nComputing Metrics...")
    f1, precision, recall = compute_f1_score(pred_aligned, gt_norm)
    iou = compute_voxel_iou(pred_aligned, gt_norm)
    chamfer = compute_chamfer_distance(pred_aligned, gt_norm)
    emd = compute_emd(pred_aligned, gt_norm, max_points=2048)
    
    print("-" * 30)
    print(f"Results for NIfTI Label {target_label_id} vs PLY")
    print("-" * 30)
    print(f"  F1@0.01:   {f1:.4f}")
    print(f"  Voxel-IoU: {iou:.4f}")
    print(f"  Chamfer:   {chamfer:.4f}")
    print(f"  EMD:       {emd:.4f}")
    print("-" * 30)

# --- Example Usage ---
mask_file = 'segmentation.nii.gz'
mesh_file = 'ground_truth.ply'
    
# Run comparison for label 1 inside the NIfTI file
evaluate_nii_vs_ply(mask_file, mesh_file, target_label_id=1)

In [1]:
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import os

# ==========================================
# 1. Metric Helper Functions (Preserved)
# ==========================================

def normalize_to_unit_cube(points):
    """Normalizes point cloud to [-1, 1] range."""
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns source points to target points using ICP."""
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
    
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    """Computes F1@0.01 score."""
    if len(pred_points) == 0 or len(gt_points) == 0: return 0.0, 0.0, 0.0
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0: return 0.0, precision, recall
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-IoU at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    return intersection / union if union > 0 else 0.0

def compute_voxel_dice(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-Dice at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    denom = len(voxels_1) + len(voxels_2)
    
    return (2.0 * intersection) / denom if denom > 0 else 0.0

def compute_chamfer_distance(pred_points, gt_points):
    """Computes Chamfer Distance (CD)."""
    if len(pred_points) == 0 or len(gt_points) == 0: return float('inf')
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    
    return np.mean(dist_pred_to_gt) + np.mean(dist_gt_to_pred)

def compute_emd(pred_points, gt_points, max_points=2048):
    """Computes Earth Mover's Distance (EMD)."""
    if len(pred_points) == 0 or len(gt_points) == 0:
        return float('inf')

    n_points = min(len(pred_points), len(gt_points), max_points)
    
    np.random.seed(42)
    indices_pred = np.random.choice(len(pred_points), n_points, replace=False)
    indices_gt = np.random.choice(len(gt_points), n_points, replace=False)
    
    pred_sampled = pred_points[indices_pred]
    gt_sampled = gt_points[indices_gt]
    
    dists = cdist(pred_sampled, gt_sampled, metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(dists)
    
    total_cost = dists[row_ind, col_ind].sum()
    emd_value = total_cost / n_points
    
    return emd_value

# ==========================================
# 2. Data Extraction Helpers (Streamlined)
# ==========================================

def load_and_sample_ply(ply_input, num_points=10000):
    """
    Loads a PLY file (or accepts an Open3D object) and samples points uniformly.
    """
    # 1. Handle string path input
    if isinstance(ply_input, str):
        if not os.path.exists(ply_input):
            print(f"Error: PLY file not found at {ply_input}")
            return np.zeros((0, 3))
        
        try:
            # Try loading as a Mesh first (most likely for surfaces)
            mesh = o3d.io.read_triangle_mesh(ply_input)
            if len(mesh.triangles) > 0:
                pcd = mesh.sample_points_uniformly(number_of_points=num_points)
                return np.asarray(pcd.points)
            else:
                # If no triangles, load as PointCloud
                pcd = o3d.io.read_point_cloud(ply_input)
                # Downsample if needed
                if len(pcd.points) > num_points:
                    # Random downsample returns a point cloud, we grab indices or just use points
                    # open3d random_down_sample ratio is float 0-1
                    ratio = num_points / len(pcd.points)
                    pcd = pcd.random_down_sample(ratio)
                return np.asarray(pcd.points)
                
        except Exception as e:
            print(f"PLY Loading Error: {e}")
            return np.zeros((0, 3))

    # 2. Handle existing Open3D objects (in-memory)
    elif isinstance(ply_input, (o3d.geometry.TriangleMesh, o3d.geometry.PointCloud)):
        try:
            if isinstance(ply_input, o3d.geometry.TriangleMesh):
                pcd = ply_input.sample_points_uniformly(number_of_points=num_points)
                return np.asarray(pcd.points)
            else:
                # It is a PointCloud
                if len(ply_input.points) > num_points:
                    ratio = num_points / len(ply_input.points)
                    pcd = ply_input.random_down_sample(ratio)
                    return np.asarray(pcd.points)
                return np.asarray(ply_input.points)
        except Exception as e:
            print(f"Object Processing Error: {e}")
            return np.zeros((0, 3))
            
    else:
        print("Error: Input must be a file path string or an Open3D geometry object.")
        return np.zeros((0, 3))

# ==========================================
# 3. Main Comparison Function
# ==========================================

def evaluate_ply_vs_ply(pred_ply, gt_ply, num_points=10000, do_icp=True):
    """
    Compares two PLY inputs (Prediction vs Ground Truth).
    
    Args:
        pred_ply: Path to prediction .ply file OR Open3D geometry object.
        gt_ply: Path to ground truth .ply file OR Open3D geometry object.
        num_points: Number of points to sample for evaluation.
        do_icp: Boolean to enable/disable ICP alignment.
    """
    print(f"Comparing Prediction PLY vs Ground Truth PLY...")
    
    # 1. Extraction
    print("Loading Prediction surface...")
    pred_points = load_and_sample_ply(pred_ply, num_points=num_points)
    
    print("Loading Ground Truth surface...")
    gt_points = load_and_sample_ply(gt_ply, num_points=num_points)
    
    # Handle Missing/Empty
    if len(pred_points) == 0 or len(gt_points) == 0:
        print("Error: One of the inputs yielded 0 points.")
        return
        
    print(f"Loaded points -> Pred: {len(pred_points)}, GT: {len(gt_points)}")

    # 2. Normalize & Align
    # Normalize both to unit cube to make metrics scale-invariant
    gt_norm = normalize_to_unit_cube(gt_points)
    pred_norm = normalize_to_unit_cube(pred_points)
    
    if do_icp:
        print("Applying ICP alignment...")
        pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
    else:
        print("Skipping ICP...")
        pred_aligned = pred_norm
    
    # 3. Compute Metrics
    print("\nComputing Metrics...")
    f1, precision, recall = compute_f1_score(pred_aligned, gt_norm)
    iou = compute_voxel_iou(pred_aligned, gt_norm)
    dice = compute_voxel_dice(pred_aligned, gt_norm)
    chamfer = compute_chamfer_distance(pred_aligned, gt_norm)
    emd = compute_emd(pred_aligned, gt_norm, max_points=2048)
    
    print("-" * 30)
    print(f"Results: Prediction vs Ground Truth")
    print("-" * 30)
    print(f"  F1@0.01:   {f1:.4f}")
    print(f"  Voxel-IoU: {iou:.4f}")
    print(f"  Voxel-Dice:{dice:.4f}")
    print(f"  Chamfer:   {chamfer:.4f}")
    print(f"  EMD:       {emd:.4f}")
    print("-" * 30)

# --- Example Usage ---
# if __name__ == "__main__":
pred_file = '/PHShome/yl535/project/python/sam_3d/sam-3d-objects/results/gaussians/RET002OD_cup.ply'
gt_file = '/PHShome/yl535/project/python/sam_3d/sam-3d-objects/results/gaussians/RET002OD_disc.ply'

# Ensure files exist before running example, or just run the function
evaluate_ply_vs_ply(pred_file, gt_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Comparing Prediction PLY vs Ground Truth PLY...
Loading Prediction surface...
[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Loading Ground Truth surface...
[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Loaded points -> Pred: 10000, GT: 10000
Applying ICP alignment...

Computing Metrics...
------------------------------
Results: Prediction vs Ground Truth
------------------------------
  F1@0.01:   0.1463
  Voxel-IoU: 0.2661
  Voxel-Dice:0.4204
  Chamfer:   0.0812
  EMD:       0.0919
------------------------------


In [ ]:
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import os
import glob
import time

# ==========================================
# 1. Metric Helper Functions
# ==========================================

def normalize_to_unit_cube(points):
    """Normalizes point cloud to [-1, 1] range."""
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns source points to target points using ICP."""
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
    
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    # Using point-to-point for general robustness without normals
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_metrics_pair(p1, p2, threshold=0.01, resolution=64, max_emd_points=2048):
    """
    Computes all 5 metrics for a single pair of point clouds (normalized).
    Returns a dictionary of results.
    """
    results = {}
    
    # --- F1 Score ---
    gt_tree = cKDTree(p2)
    pred_tree = cKDTree(p1)
    
    dist_p1_to_p2, _ = gt_tree.query(p1, k=1)
    precision = np.mean(dist_p1_to_p2 < threshold)
    
    dist_p2_to_p1, _ = pred_tree.query(p2, k=1)
    recall = np.mean(dist_p2_to_p1 < threshold)
    
    if precision + recall == 0:
        results['f1'] = 0.0
    else:
        results['f1'] = 2 * (precision * recall) / (precision + recall)

    # --- Voxel IoU & Dice ---
    min_bound = np.array([-1.0, -1.0, -1.0])
    voxel_size = 2.0 / resolution # range [-1, 1] is size 2
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(p1)
    voxels_2 = get_voxels(p2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    denom = len(voxels_1) + len(voxels_2)
    
    results['iou'] = intersection / union if union > 0 else 0.0
    results['dice'] = (2.0 * intersection) / denom if denom > 0 else 0.0

    # --- Chamfer Distance ---
    # (Symmetric Chamfer: mean(d(p1->p2)) + mean(d(p2->p1)))
    results['chamfer'] = np.mean(dist_p1_to_p2) + np.mean(dist_p2_to_p1)

    # --- EMD ---
    n_points = min(len(p1), len(p2), max_emd_points)
    # Simple random sampling for EMD speed
    # Note: For strict determinism, set seed inside or pass seed
    idx1 = np.random.choice(len(p1), n_points, replace=False)
    idx2 = np.random.choice(len(p2), n_points, replace=False)
    
    dists = cdist(p1[idx1], p2[idx2], metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(dists)
    results['emd'] = dists[row_ind, col_ind].sum() / n_points

    return results

# ==========================================
# 2. Data Loading Helpers
# ==========================================

def load_and_process_ply(path, num_points=10000):
    """Loads a PLY, samples it, and normalizes it to unit cube."""
    try:
        # Load Mesh or PCD
        # Try mesh first
        mesh = o3d.io.read_triangle_mesh(path)
        if len(mesh.triangles) > 0:
            pcd = mesh.sample_points_uniformly(number_of_points=num_points)
            points = np.asarray(pcd.points)
        else:
            pcd = o3d.io.read_point_cloud(path)
            # Basic fallback sampling
            if len(pcd.points) > 0:
                if len(pcd.points) > num_points:
                    pcd = pcd.random_down_sample(num_points / len(pcd.points))
                points = np.asarray(pcd.points)
            else:
                return None

        return normalize_to_unit_cube(points)
    except Exception as e:
        print(f"Failed to load {path}: {e}")
        return None

def load_group(file_paths, num_points=10000):
    """Loads a list of file paths into memory as normalized numpy arrays."""
    data = []
    print(f"Loading {len(file_paths)} files...")
    for f in file_paths:
        pts = load_and_process_ply(f, num_points)
        if pts is not None and len(pts) > 0:
            data.append(pts)
    return data

# ==========================================
# 3. Energy Distance Logic
# ==========================================

def calculate_energy_distance(group_A, group_B, do_icp=True):
    """
    Computes Energy Distance between two groups of shapes (Group A and Group B).
    
    Energy Distance D^2(A, B) = 2*E[d(a,b)] - E[d(a,a')] - E[d(b,b')]
    
    For similarity metrics (F1, IoU, Dice), we convert to distance: d = 1 - score.
    For distance metrics (Chamfer, EMD), we use raw values.
    """
    N = len(group_A)
    M = len(group_B)
    
    if N == 0 or M == 0:
        print("Error: One of the groups is empty.")
        return None

    # Initialize accumulation matrices
    # We will store: sum, count to calculate means later
    keys = ['f1', 'iou', 'dice', 'chamfer', 'emd']
    
    # Structures to hold raw distances for each component of Energy Distance
    # d_XY represents distances between Group A and Group B
    # d_XX represents distances within Group A
    # d_YY represents distances within Group B
    d_XY = {k: [] for k in keys}
    d_XX = {k: [] for k in keys}
    d_YY = {k: [] for k in keys}

    print("\n--- Computing Cross-Distances (A vs B) ---")
    # 1. Compute Cross-Term (A vs B)
    # We compute all N*M pairs
    count = 0
    total_pairs = N * M
    for i, p_a in enumerate(group_A):
        for j, p_b in enumerate(group_B):
            if count % 10 == 0: print(f"  Processing pair {count}/{total_pairs}...", end='\r')
            
            # Align A to B (or vice versa)
            if do_icp:
                p_a_aligned = apply_icp(p_a, p_b)
            else:
                p_a_aligned = p_a
                
            res = compute_metrics_pair(p_a_aligned, p_b)
            
            # Store values
            # Convert Similarity -> Distance
            d_XY['f1'].append(1.0 - res['f1'])
            d_XY['iou'].append(1.0 - res['iou'])
            d_XY['dice'].append(1.0 - res['dice'])
            # Keep Distances as is
            d_XY['chamfer'].append(res['chamfer'])
            d_XY['emd'].append(res['emd'])
            count += 1

    print("\n--- Computing Self-Distances (A vs A) ---")
    # 2. Compute Self-Term (A vs A)
    # We only need upper triangle for distinct pairs, but Energy Distance usually 
    # implies full sum. d(x,x) is 0.
    count = 0
    # Optimization: Only loop i < j (since dist(a,b) ~ dist(b,a)) 
    # and assume dist(a,a) = 0.
    # However, strictly speaking, ICP is not perfectly symmetric. 
    # For rigorousness, we do full loops or assume symmetry. 
    # Let's assume symmetry to save 50% time: d(i,j) == d(j,i).
    for i in range(N):
        for j in range(i + 1, N):
            if do_icp:
                p_i_aligned = apply_icp(group_A[i], group_A[j])
            else:
                p_i_aligned = group_A[i]

            res = compute_metrics_pair(p_i_aligned, group_A[j])
            
            val_f1 = 1.0 - res['f1']
            val_iou = 1.0 - res['iou']
            val_dice = 1.0 - res['dice']
            val_cd = res['chamfer']
            val_emd = res['emd']
            
            # Add twice (for i,j and j,i)
            d_XX['f1'].extend([val_f1, val_f1])
            d_XX['iou'].extend([val_iou, val_iou])
            d_XX['dice'].extend([val_dice, val_dice])
            d_XX['chamfer'].extend([val_cd, val_cd])
            d_XX['emd'].extend([val_emd, val_emd])
    
    # Add zeros for the diagonal (i=i) if we want technically correct means over N*N
    # N zeros for each metric
    for k in keys:
        d_XX[k].extend([0.0] * N)

    print("\n--- Computing Self-Distances (B vs B) ---")
    # 3. Compute Self-Term (B vs B)
    for i in range(M):
        for j in range(i + 1, M):
            if do_icp:
                p_i_aligned = apply_icp(group_B[i], group_B[j])
            else:
                p_i_aligned = group_B[i]

            res = compute_metrics_pair(p_i_aligned, group_B[j])
            
            val_f1 = 1.0 - res['f1']
            val_iou = 1.0 - res['iou']
            val_dice = 1.0 - res['dice']
            val_cd = res['chamfer']
            val_emd = res['emd']
            
            d_YY['f1'].extend([val_f1, val_f1])
            d_YY['iou'].extend([val_iou, val_iou])
            d_YY['dice'].extend([val_dice, val_dice])
            d_YY['chamfer'].extend([val_cd, val_cd])
            d_YY['emd'].extend([val_emd, val_emd])
            
    for k in keys:
        d_YY[k].extend([0.0] * M)

    # 4. Final Calculation
    energy_results = {}
    print("\n" + "="*40)
    print(f"ENERGY DISTANCE RESULTS (A: {N} files, B: {M} files)")
    print("Interpretation: Lower is more similar distributions.")
    print("For F1/IoU/Dice, metric used was (1 - Score).")
    print("="*40)

    for k in keys:
        # Calculate Expectation (Mean)
        E_XY = np.mean(d_XY[k])
        E_XX = np.mean(d_XX[k])
        E_YY = np.mean(d_YY[k])
        
        # Energy Distance Formula
        ed_value = 2 * E_XY - E_XX - E_YY
        
        energy_results[k] = ed_value
        print(f"{k.upper():<10} | Energy Dist: {ed_value:.6f} (Avg Cross: {E_XY:.4f}, Avg Intra-A: {E_XX:.4f}, Avg Intra-B: {E_YY:.4f})")

    return energy_results

# ==========================================
# 4. Main Execution
# ==========================================

if __name__ == "__main__":
    # --- Configuration ---
    # Define directories containing PLY files
    # Example paths - replace with your actual folders
    group_a_dir = "./data/group_a_shapes"
    group_b_dir = "./data/group_b_shapes"
    
    # Or manual lists
    # group_a_files = ['a1.ply', 'a2.ply']
    # group_b_files = ['b1.ply', 'b2.ply', 'b3.ply']
    
    # Just creating dummy data for demonstration if folders don't exist
    if not os.path.exists(group_a_dir):
        print("Note: Example directories not found. Creating dummy spheres for demo...")
        # Create dummy data
        def make_sphere(radius, offset):
            mesh = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
            mesh.translate(offset)
            return mesh.sample_points_uniformly(1000)
            
        # Group A: 3 Small spheres
        data_A = [np.asarray(make_sphere(1.0, [0,0,0]).points) for _ in range(3)]
        # Group B: 4 Slightly larger spheres (N != M)
        data_B = [np.asarray(make_sphere(1.1, [0.1,0,0]).points) for _ in range(4)]
        
        # Normalize them as the loader usually would
        data_A = [normalize_to_unit_cube(p) for p in data_A]
        data_B = [normalize_to_unit_cube(p) for p in data_B]
    else:
        # Real Loading
        files_A = glob.glob(os.path.join(group_a_dir, "*.ply"))
        files_B = glob.glob(os.path.join(group_b_dir, "*.ply"))
        data_A = load_group(files_A)
        data_B = load_group(files_B)

    # Run Evaluation
    if len(data_A) > 0 and len(data_B) > 0:
        start_time = time.time()
        results = calculate_energy_distance(data_A, data_B, do_icp=True)
        print(f"\nTotal calculation time: {time.time() - start_time:.2f} seconds")
    else:
        print("No data loaded.")

In [ ]:
import numpy as np
import open3d as o3d
import nibabel as nib
from skimage import measure  # For Marching Cubes
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import os

# ==========================================
# 1. Metric Helper Functions (Preserved)
# ==========================================

def normalize_to_unit_cube(points):
    """Normalizes point cloud to [-1, 1] range."""
    if len(points) == 0: return points
    centroid = np.mean(points, axis=0)
    points_centered = points - centroid
    max_dist = np.max(np.abs(points_centered))
    return points_centered / max_dist if max_dist > 0 else points_centered

def apply_icp(source_points, target_points, threshold=0.02):
    """Aligns source points to target points using ICP."""
    if len(source_points) == 0 or len(target_points) == 0:
        return source_points
    
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_points)
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_points)
    
    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    source_pcd.transform(reg_p2p.transformation)
    return np.asarray(source_pcd.points)

def compute_f1_score(pred_points, gt_points, threshold=0.01):
    """Computes F1@0.01 score."""
    if len(pred_points) == 0 or len(gt_points) == 0: return 0.0, 0.0, 0.0
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    precision = np.mean(dist_pred_to_gt < threshold)
    
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    recall = np.mean(dist_gt_to_pred < threshold)
    
    if precision + recall == 0: return 0.0, precision, recall
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_voxel_iou(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-IoU at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    union = len(voxels_1.union(voxels_2))
    return intersection / union if union > 0 else 0.0

def compute_voxel_dice(point_cloud_1, point_cloud_2, resolution=64):
    """Computes Voxel-Dice at 64^3 resolution."""
    if len(point_cloud_1) == 0 and len(point_cloud_2) == 0: return 1.0
    if len(point_cloud_1) == 0 or len(point_cloud_2) == 0: return 0.0
    
    min_bound = np.array([-1.0, -1.0, -1.0])
    max_bound = np.array([1.0, 1.0, 1.0])
    voxel_size = (max_bound - min_bound) / resolution
    
    def get_voxels(points):
        indices = np.floor((points - min_bound) / voxel_size).astype(int)
        indices = np.clip(indices, 0, resolution - 1)
        return set(map(tuple, indices))

    voxels_1 = get_voxels(point_cloud_1)
    voxels_2 = get_voxels(point_cloud_2)
    
    intersection = len(voxels_1.intersection(voxels_2))
    denom = len(voxels_1) + len(voxels_2)
    
    return (2.0 * intersection) / denom if denom > 0 else 0.0

def compute_chamfer_distance(pred_points, gt_points):
    """Computes Chamfer Distance (CD)."""
    if len(pred_points) == 0 or len(gt_points) == 0: return float('inf')
    
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    dist_pred_to_gt, _ = gt_tree.query(pred_points, k=1)
    dist_gt_to_pred, _ = pred_tree.query(gt_points, k=1)
    
    return np.mean(dist_pred_to_gt) + np.mean(dist_gt_to_pred)

def compute_emd(pred_points, gt_points, max_points=2048):
    """Computes Earth Mover's Distance (EMD)."""
    if len(pred_points) == 0 or len(gt_points) == 0:
        return float('inf')

    n_points = min(len(pred_points), len(gt_points), max_points)
    
    np.random.seed(42)
    indices_pred = np.random.choice(len(pred_points), n_points, replace=False)
    indices_gt = np.random.choice(len(gt_points), n_points, replace=False)
    
    pred_sampled = pred_points[indices_pred]
    gt_sampled = gt_points[indices_gt]
    
    dists = cdist(pred_sampled, gt_sampled, metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(dists)
    
    total_cost = dists[row_ind, col_ind].sum()
    emd_value = total_cost / n_points
    
    return emd_value

# ==========================================
# 2. Data Extraction Helpers (Updated for NII)
# ==========================================

def load_and_sample_ply(ply_input, num_points=10000):
    """Loads a PLY file and samples points."""
    if isinstance(ply_input, str):
        if not os.path.exists(ply_input):
            print(f"Error: PLY file not found at {ply_input}")
            return np.zeros((0, 3))
        try:
            mesh = o3d.io.read_triangle_mesh(ply_input)
            if len(mesh.triangles) > 0:
                pcd = mesh.sample_points_uniformly(number_of_points=num_points)
                return np.asarray(pcd.points)
            else:
                pcd = o3d.io.read_point_cloud(ply_input)
                if len(pcd.points) > num_points:
                    ratio = num_points / len(pcd.points)
                    pcd = pcd.random_down_sample(ratio)
                return np.asarray(pcd.points)
        except Exception as e:
            print(f"PLY Loading Error: {e}")
            return np.zeros((0, 3))
    return np.zeros((0, 3))

def load_nii_as_surface_points(nii_path, num_points=10000, iso_value=0.5):
    """
    Loads a NIfTI file, extracts the isosurface (Marching Cubes), 
    and samples points from that surface.
    """
    if not os.path.exists(nii_path):
        print(f"Error: NIfTI file not found at {nii_path}")
        return np.zeros((0, 3))
    
    try:
        # 1. Load NIfTI
        nii = nib.load(nii_path)
        data = nii.get_fdata()
        
        # 2. Check if empty
        if data.max() < iso_value:
            print("Warning: NIfTI volume is empty (all values below threshold).")
            return np.zeros((0, 3))

        # 3. Marching Cubes to get surface vertices (in index coordinates)
        # We assume the mask is binary or has a clear threshold
        verts, faces, _, _ = measure.marching_cubes(data, level=iso_value)
        
        # 4. Apply voxel spacing (zooms) to preserve aspect ratio
        # We do NOT apply full affine (translation/rotation) usually, because
        # we are going to normalize to unit cube anyway. But scaling matters.
        zooms = nii.header.get_zooms()[:3]
        verts = verts * np.array(zooms)
        
        # 5. Convert to Open3D Mesh for easy sampling
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(verts)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        
        # 6. Sample Points
        pcd = mesh.sample_points_uniformly(number_of_points=num_points)
        return np.asarray(pcd.points)

    except Exception as e:
        print(f"NIfTI Processing Error: {e}")
        return np.zeros((0, 3))

# ==========================================
# 3. Main Comparison Function (PLY vs NII)
# ==========================================

def evaluate_ply_vs_nii(pred_ply_path, gt_nii_path, num_points=10000, do_icp=True):
    """
    Compares a generated PLY against a Ground Truth NIfTI.
    """
    print(f"\n--- Comparing Prediction (PLY) vs GT (NIfTI) ---")
    print(f"Pred: {os.path.basename(pred_ply_path)}")
    print(f"GT:   {os.path.basename(gt_nii_path)}")
    
    # 1. Extraction
    print("Loading Prediction PLY...")
    pred_points = load_and_sample_ply(pred_ply_path, num_points=num_points)
    
    print("Extracting Surface from GT NIfTI...")
    gt_points = load_nii_as_surface_points(gt_nii_path, num_points=num_points)
    
    # Handle Missing/Empty
    if len(pred_points) == 0 or len(gt_points) == 0:
        print("Error: One of the inputs yielded 0 points.")
        return
        
    print(f"Loaded points -> Pred: {len(pred_points)}, GT (Surface): {len(gt_points)}")

    # 2. Normalize 
    # This is crucial because NIfTI is in mm/voxels and PLY might be in arbitrary units
    gt_norm = normalize_to_unit_cube(gt_points)
    pred_norm = normalize_to_unit_cube(pred_points)
    
    # 3. Align (ICP)
    if do_icp:
        print("Applying ICP alignment...")
        # Align Pred to GT
        pred_aligned = apply_icp(pred_norm, gt_norm, threshold=0.05)
    else:
        pred_aligned = pred_norm
    
    # 4. Compute Metrics
    print("Computing Metrics...")
    f1, precision, recall = compute_f1_score(pred_aligned, gt_norm)
    iou = compute_voxel_iou(pred_aligned, gt_norm)
    dice = compute_voxel_dice(pred_aligned, gt_norm)
    chamfer = compute_chamfer_distance(pred_aligned, gt_norm)
    emd = compute_emd(pred_aligned, gt_norm, max_points=2048)
    
    print("-" * 30)
    print(f"Results: {os.path.basename(pred_ply_path)} vs {os.path.basename(gt_nii_path)}")
    print("-" * 30)
    print(f"  F1@0.01:   {f1:.4f}")
    print(f"  Voxel-IoU: {iou:.4f}")
    print(f"  Voxel-Dice:{dice:.4f}")
    print(f"  Chamfer:   {chamfer:.4f}")
    print(f"  EMD:       {emd:.4f}")
    print("-" * 30)

# --- Example Usage ---
if __name__ == "__main__":
    # Update these paths to your actual files
    pred_ply = '/PHShome/yl535/project/python/sam_3d/sam-3d-objects/results_AeroPath/lungs/10_CT_HR_1.nii.gz.ply'
    gt_nii   = '/PHShome/yl535/project/python/datasets/AeroPath/lungs_segmented/10_CT_HR_1_mask.nii.gz'

    # Check if files exist before running to avoid immediate errors
    if os.path.exists(pred_ply) and os.path.exists(gt_nii):
        evaluate_ply_vs_nii(pred_ply, gt_nii)
    else:
        print("Please set valid paths in the __main__ block.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

--- Comparing Prediction (PLY) vs GT (NIfTI) ---
Pred: 10_CT_HR_1.nii.gz.ply
GT:   10_CT_HR_1.nii.gz
Loading Prediction PLY...
[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Extracting Surface from GT NIfTI...
Error: One of the inputs yielded 0 points.


In [2]:
import nibabel as nib
import open3d as o3d
import numpy as np
import os

def inspect_nifti(nii_path):
    print(f"\n--- Inspecting NIfTI: {os.path.basename(nii_path)} ---")
    if not os.path.exists(nii_path):
        print(f"❌ File not found at: {nii_path}")
        return

    try:
        nii = nib.load(nii_path)
        data = nii.get_fdata()
        
        print(f"  Shape: {data.shape}")
        print(f"  Data Type: {data.dtype}")
        print(f"  Min Value: {data.min()}")
        print(f"  Max Value: {data.max()}")
        print(f"  Unique Values (first 10): {np.unique(data)[:10]}")
        
        # Check for empty mask
        if data.max() <= 0:
            print("❌ ERROR: Max value is <= 0. The volume appears empty.")
        elif data.max() < 0.5:
            print(f"⚠️  WARNING: Max value ({data.max()}) is < 0.5. 'marching_cubes' at level=0.5 will fail.")
        else:
            print("✅ Data looks valid for isosurface extraction.")
            
    except Exception as e:
        print(f"❌ Error loading NIfTI: {e}")

def inspect_ply(ply_path):
    print(f"\n--- Inspecting PLY: {os.path.basename(ply_path)} ---")
    if not os.path.exists(ply_path):
        print(f"❌ File not found at: {ply_path}")
        return

    try:
        # Attempt 1: Load as Mesh (requires triangles)
        mesh = o3d.io.read_triangle_mesh(ply_path)
        if mesh.has_vertices():
            print(f"  [As Mesh] Vertices: {len(mesh.vertices)}")
            print(f"  [As Mesh] Triangles: {len(mesh.triangles)}")
            if len(mesh.triangles) == 0:
                print("⚠️  WARNING: No triangles found. This is likely a Point Cloud, not a Mesh.")
        else:
            print("  [As Mesh] Failed to load vertices.")

        # Attempt 2: Load as Point Cloud
        pcd = o3d.io.read_point_cloud(ply_path)
        if pcd.has_points():
            print(f"  [As PCD]  Points: {len(pcd.points)}")
            if len(pcd.points) > 0:
                print("✅ PLY contains valid points.")
        else:
            print("❌ ERROR: PLY appears empty (0 points).")

    except Exception as e:
        print(f"❌ Error loading PLY: {e}")

# --- UPDATE THESE PATHS ---
bad_pred_ply = '/PHShome/yl535/project/python/sam_3d/sam-3d-objects/results_AeroPath/lungs/10_CT_HR_1.nii.gz.ply'
bad_gt_nii   = '/PHShome/yl535/project/python/datasets/AeroPath/lungs_segmented/10_CT_HR_1.nii.gz'



inspect_ply(bad_pred_ply)
inspect_nifti(bad_gt_nii)


--- Inspecting PLY: 10_CT_HR_1.nii.gz.ply ---
[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
  [As Mesh] Vertices: 206720
  [As Mesh] Triangles: 0
⚠️  WARNING: No triangles found. This is likely a Point Cloud, not a Mesh.
  [As PCD]  Points: 206720
✅ PLY contains valid points.

--- Inspecting NIfTI: 10_CT_HR_1.nii.gz ---
  Shape: (485, 337, 219)
  Data Type: float64
  Min Value: -1024.0
  Max Value: 0.0
  Unique Values (first 10): [-1024. -1023. -1022. -1021. -1020. -1019. -1018. -1017. -1016. -1015.]
❌ ERROR: Max value is <= 0. The volume appears empty.
